Recover working example of cross section tuning:

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import mceq_config as config
from MCEq.core import MCEqRun
import crflux.models as pm
from MCEq.data import InteractionCrossSections
import scienceplots
import matplotlib

import os
os.chdir('/home/khymon/scripts/master/SeasonalVariationUnfolding/')
import HelperFunctions as un
os.chdir('/home/khymon/scripts/SeasonalVariationsMCEq/')

In [14]:
#initialize mceq instances

mceq = MCEqRun(
    interaction_model="SIBYLL2.3c",
    theta_deg=0.0,
    primary_model=(pm.HillasGaisser2012, "H3a"),
    density_model = (('MSIS00_IC',('SouthPole','January')))
)
mceq_tune = MCEqRun(
    interaction_model="SIBYLL2.3c",
    theta_deg=0.0,
    primary_model=(pm.HillasGaisser2012, "H3a"),
    density_model = (('MSIS00_IC',('SouthPole','January')))
)

MCEqRun::set_interaction_model(): SIBYLL23C
ParticleManager::_init_default_tracking(): Initializing default tracking categories (pi, K, mu)
MCEqRun::set_density_model(): Setting density profile to MSIS00_IC ('SouthPole', 'January')
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00
MCEqRun::set_primary_model(): HillasGaisser2012 H3a
MCEqRun::set_interaction_model(): SIBYLL23C
ParticleManager::_init_default_tracking(): Initializing default tracking categories (pi, K, mu)
MCEqRun::set_density_model(): Setting density profile to MSIS00_IC ('SouthPole', 'January')
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00
MCEqRun::set_primary_model(): HillasGaisser2012 H3a


In [15]:
# class for modifying cross section in MCEq
# more functions can be defined as in example function mod1

class ModIntCrossSections(InteractionCrossSections):
    def __init__(self, mceq_hdf_db, interaction_model="SIBYLL2.3c", modmod=0, scale_factor=0.025):
        self.modmod = modmod  # Modification mode
        self.scale_factor = scale_factor  # Default scaling factor
        super().__init__(mceq_hdf_db, interaction_model)  # Call parent constructor

    def mod1(self, scale_factor=None):
        """ Pion modification above 200GeV - exp increase """
        if scale_factor is None:
            scale_factor = self.scale_factor  # Use the default if not specified
        e_range = self.energy_grid.c > 100.0
        e0 = self.energy_grid.c[e_range][0]
        print(f"Applying mod1 with scaling factor: {scale_factor}, e0: {e0}")
        for p in [211]:  # Loop over particle types # modify pions
            self.index_d[p][e_range] *= (self.energy_grid.c[e_range] / e0) ** scale_factor

    def mod2(self, scale_factor=None):
        """ Kaon modification above 200GeV - exp increase """
        if scale_factor is None:
            scale_factor = self.scale_factor
        e_range = self.energy_grid.c > 200.0
        e0 = self.energy_grid.c[e_range][0]
        print(f"Applying mod2 with scaling factor: {scale_factor}, e0: {e0}")
        for p in [321]: # modify kaons
            self.index_d[p][e_range] *= (self.energy_grid.c[e_range] / e0) ** scale_factor  # Different scaling logic
    
    def mod3(self, scale_factor=None):
        """ Pion modification above 200GeV - constant off set """
        if scale_factor is None:
            scale_factor = self.scale_factor
        e_range = self.energy_grid.c > 200.0
        e0 = self.energy_grid.c[e_range][0]
 
        print(f"Applying mod2 with scaling factor: {scale_factor}, e0: {e0}")
        for p in [211]: # modify kaons
            self.index_d[p][e_range] *= (scale_factor)

    def mod4(self, scale_factor=None):
        """ Kaon modification above 200GeV - constant off set """
        if scale_factor is None:
            scale_factor = self.scale_factor
        e_range = self.energy_grid.c > 200.0
        e0 = self.energy_grid.c[e_range][0]
 
        print(f"Applying mod2 with scaling factor: {scale_factor}, e0: {e0}")
        for p in [321]: # modify kaons
            self.index_d[p][e_range] *= (scale_factor)

    def load(self, interaction_model):
        """ Load the interaction model and apply the modification function based on mode - add more in the same sty;e """
        super().load(interaction_model)
        if self.modmod == 1:
            self.mod1()
        elif self.modmod == 2:
            self.mod2()
        elif self.modmod == 3:
            self.mod3()
        elif self.modmod == 4:
            self.mod4()


In [16]:
def runmceq(mceq_object,ptype,doys,angles): #create 
    '''
        Calculation of MCEq

        ptype: string (numu, nue or mu)
        mceq_object: mceq object with or without modified settings. MSIS00 atmosphere is required
        doys: list (int) of days in year from 1 to 366
        angles: list (float), supported angles between 0 and 180
    '''

    mag = 0
    flux = []#np.ndarray(shape=(len(angles),len(doys),121))

    for sim_angle in angles: # for just one zenith band [ic79_Aeff[0]] - wrong zenith - just proof-of-cenpt
        
        mceq_object.set_theta_deg(sim_angle)

        flux.append([])
        for d in doys:
            mceq_object.density_model.set_doy(d)
            mceq_object.density_model.calculate_density_spline()
            mceq_object._calculate_integration_path(int_grid=None, grid_var='X',force=True)
            mceq_object.solve()
            if ptype == 'numu':
                flux[-1].append(mceq_object.get_solution('total_numu', mag) + mceq_object.get_solution('total_antinumu' ,mag))
            elif ptype == 'nue':
                flux[-1].append(mceq_object.get_solution('total_nue', mag) + mceq_object.get_solution('total_antinue' ,mag))
            elif ptype == 'mu':
                flux[-1].append(mceq_object.get_solution('total_mu+', mag) + mceq_object.get_solution('total_mu-' ,mag))
            else:
                print('particle type is not defined.')
                exit()

    return np.array(flux)

In [17]:
# apply a modification

modcs4 = ModIntCrossSections(mceq._mceq_db, interaction_model="SIBYLL2.3c", modmod=4, scale_factor=1.3)
modcs4.load(interaction_model="SIBYLL2.3c")

mceq_tune._int_cs = modcs4 # add modification to cross section in mceq instance
mceq_tune.set_interaction_model("SIBYLL2.3c", force=True) # necessary to force cross section change


#test if tuning is correct:
print('ratio pion cross section tuned/untuned: ', modcs4.get_cs(211, mbarn=True)/InteractionCrossSections(mceq._mceq_db, interaction_model="SIBYLL2.3c").get_cs(211, mbarn=True))
print('ratio pion cross section tuned/untuned: ', modcs4.get_cs(321, mbarn=True)/InteractionCrossSections(mceq._mceq_db, interaction_model="SIBYLL2.3c").get_cs(321, mbarn=True))

Applying mod2 with scaling factor: 1.3, e0: 223.872113856834
Applying mod2 with scaling factor: 1.3, e0: 223.872113856834
MCEqRun::set_interaction_model(): SIBYLL23C
Applying mod2 with scaling factor: 1.3, e0: 223.872113856834


MCEqRun::set_primary_model(): HillasGaisser2012 H3a
ratio pion cross section tuned/untuned:  [nan nan nan nan nan nan nan nan  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
ratio pion cross section tuned/untuned:  [nan nan nan nan nan nan nan nan 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.3 1.3
 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3
 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3
 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3
 1.3 1.3 1

/home/khymon/software/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/home/khymon/software/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [18]:
# definition of angles and days

max_zenith = np.cos(np.deg2rad(100.)) # for testing purposes only
min_zenith = np.cos(np.deg2rad(90.))
angles_edges = np.arccos(np.linspace(min_zenith,max_zenith,3))*180./np.pi # edges theta
angles = np.zeros(2)
for i in range(len(angles)):
    angles[i] = np.mean([angles_edges[i],angles_edges[i+1]])
    # calculate bin midth for theta
angles = np.round(angles,decimals=2)

# take every 5th day for fast testing purposes
doys = np.arange(1, 362, 180, dtype=int)


In [ ]:
angles = np.array([95.])
flux_untuned = runmceq(mceq,'numu',doys,angles)

MSIS00IceCubeCentered::set_theta(): latitude = -80.00 for zenith angle = 95.00
MSIS00IceCubeCentered::set_theta(): theta = 95.00 below horizon. using theta = 85.00
ratio  tuned/untuned cs pion:  [nan nan nan nan nan nan nan nan  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
ratio  tuned/untuned cs kaon:  [nan nan nan nan nan nan nan nan  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1

/home/khymon/software/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
/home/khymon/software/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [20]:
flux_tuned = runmceq(mceq_tune,'numu',doys,angles)

MSIS00IceCubeCentered::set_theta(): latitude = -85.02 for zenith angle = 92.49
MSIS00IceCubeCentered::set_theta(): theta = 92.49 below horizon. using theta = 87.51


MSIS00IceCubeCentered::set_theta(): latitude = -75.02 for zenith angle = 97.49
MSIS00IceCubeCentered::set_theta(): theta = 97.49 below horizon. using theta = 82.51


In [21]:
flux_tuned/flux_untuned

array([[[ 1.00178916,  1.0016607 ,  1.00156508,  1.00163748,
          1.00148345,  1.00129443,  1.00118409,  1.00124916,
          1.0014099 ,  1.00173321,  1.00227787,  1.00314896,
          1.00444413,  1.00624961,  1.00866342,  1.01194086,
          1.0163037 ,  1.02158626,  1.02790205,  1.03507578,
          1.04289948,  1.05108106,  1.0592277 ,  1.06690462,
          1.0737356 ,  1.07943251,  1.08387813,  1.08707338,
          1.08920519,  1.0905633 ,  1.09108891,  1.09091777,
          1.09025041,  1.08899413,  1.08766748,  1.08633385,
          1.0845792 ,  1.08224002,  1.07927445,  1.07562679,
          1.07137315,  1.06670443,  1.06183614,  1.05702095,
          1.05255238,  1.04865456,  1.04554821,  1.04324152,
          1.04173397,  1.04098583,  1.04084456,  1.04114639,
          1.04172861,  1.04241941,  1.04310074,  1.04367093,
          1.04410044,  1.04434539,  1.04432442,  1.04406564,
          1.04356854,  1.04286378,  1.04199447,  1.04092138,
          1.03967056,  1